# Character-Level LSTM in PyTorch

На этом семинаре поговорим про RNN. Здесь мы обучим модель на тексте Анны Карениной, а затем будем генерировать новый текст.**Эта модель сможет генерировать новый текст на основе нашего текста!**

Можно посмотреть [очень крутую статью про RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/), а также [не менее крутуя статью про LSTM](http://colah.github.io/posts/2015-08-Understanding-LSTMs/). Ниже представлена ​​общая архитектура RNN.

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charseq.jpeg?raw=1" width="500">

Сначала загрузим ресурсы, необходимые для загрузки данных и создания модели.

In [1]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Затем мы загрузим текстовый файл Анны Карениной.

In [2]:
# open text file and read in data as `text`
with open('anna.txt', 'r') as f:
    text = f.read()

Давайте проверим первые 100 символов, убедимся, что все красиво. 

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
len(text)

1985223

### Tokenization

В ячейках ниже постараемся создать пару **словарей** для преобразования символов в целые числа и обратно. Кодирование символов как целых чисел упрощает их использование в качестве входных данных в сети.

In [5]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

И мы можем видеть те же самые символы сверху, закодированные как целые числа.

In [6]:
encoded[:100]

array([23, 17, 14, 79, 58, 19, 10,  0, 48, 78, 78, 78, 82, 14, 79, 79, 25,
        0, 66, 14, 32, 63, 62, 63, 19, 29,  0, 14, 10, 19,  0, 14, 62, 62,
        0, 14, 62, 63,  5, 19,  3,  0, 19, 71, 19, 10, 25,  0, 77, 35, 17,
       14, 79, 79, 25,  0, 66, 14, 32, 63, 62, 25,  0, 63, 29,  0, 77, 35,
       17, 14, 79, 79, 25,  0, 63, 35,  0, 63, 58, 29,  0, 60, 73, 35, 78,
       73, 14, 25, 37, 78, 78, 27, 71, 19, 10, 25, 58, 17, 63, 35])

## Pre-processing the data

Как вы можете видеть на изображении char-RNN выше, наш LSTM ожидает ввода, который  **one-hot encoded** , что означает, что каждый символ преобразуется в целое число (через наш созданный словарь), а затем преобразуется в столбец вектор, где только соответствующий ему целочисленный индекс будет иметь значение 1, а остальная часть вектора будет заполнена нулями. Давайте создадим для этого функцию!


In [7]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [8]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


## Making training mini-batches


Для обучения на этих данных нужно создать мини-батчи для обучения. На простом примере наши батчи будут выглядеть так:

<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/sequence_batching@1x.png?raw=1" width=500px>


<br>

В этом примере возьмем закодированные символы (переданные как параметр arr) и разделим их на несколько последовательностей, заданных параметром batch_size. Каждая из наших последовательностей будет иметь длину seq_length.

### Creating Batches

**1. Первое, что нам нужно сделать, это отбросить часть текста, чтобы у нас были только полные мини-батчи.**

Каждый батч содержит $ N\times M $ символов, где $ N $ - это размер батча (количество последовательностей в батче), а $ M $ - длина seq_length или количество шагов в последовательности. Затем, чтобы получить общее количество батчей $ K $, которое мы можем сделать из массива arr, нужно разделить длину arr на количество символов в батче. Как только вы знаете количество пакетов, вы можете получить общее количество символов, которые нужно сохранить, из `arr`, $ N * M * K $.

**2. После этого нам нужно разделить arr на $N$ пакетов.** 

Вы можете сделать это с помощью `arr.reshape(size)`, где `size` - это кортеж, содержащий размеры измененного массива. Мы знаем, что нам нужно $ N $ последовательностей в батче, поэтому сделаем его размером первого измерения. Для второго измерения можем использовать «-1» в качестве заполнителя, он заполнит массив соответствующими данными. После этого должен остаться массив $N\times(M * K)$.

**3. Теперь, когда у нас есть этот массив, мы можем перебирать его, чтобы получить наши мини-батчи.**

Идея состоит в том, что каждая партия представляет собой окно $ N\times M $ в массиве $ N\times (M * K) $. Для каждого последующего батча окно перемещается на seq_length. Мы также хотим создать как входной, так и выходной массивы. Это окно можно сделать с помощью `range`, чтобы делать шаги размером `n_steps` от $ 0 $ до `arr.shape [1]`, общее количество токенов в каждой последовательности. Таким образом, целые числа, которые получены из диапазона, всегда указывают на начало батча, и каждое окно имеет ширину seq_length.

> **TODO:** Напишите код для создания батчей в функции ниже. 

In [9]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    ## TODO: Get the number of batches we can make
    batch_size_total = ...
    n_batches = ...
    
    ## TODO: Keep only enough characters to make full batches
    arr = ...
    
    ## TODO: Reshape into batch_size rows
    arr = ...

    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Test Your Implementation

Теперь создадим несколько наборов данных, и проверим, что происходит, когда мы пакетируем данные.

In [15]:
encoded[:8*100].reshape((8, -1))[:, 20:30]

array([[40, 79, 77, 79,  5, 17, 23,  4, 44,  5],
       [73, 23, 43, 33,  5, 23, 66, 14, 77, 29],
       [11, 44,  5, 73, 47, 33, 54,  7, 79, 44],
       [23, 47, 29, 13, 77, 68, 23, 73, 29, 43],
       [78, 17, 45, 23,  4, 73, 68, 23, 73, 29],
       [23,  2,  5, 44,  5, 23, 39,  4, 79, 73],
       [79, 73,  7, 23, 43, 29,  7,  5, 43, 33],
       [33, 23, 29, 73,  5, 23,  4, 73, 29, 43]])

In [18]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [19]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[22 33  4 39 43  5 44 23 32 54]
 [17 29 73 23 43 33  4 43 23  4]
 [ 5 73 68 23 29 44 23  4 23  3]
 [17 23 43 33  5 23 47 33 79  5]
 [23 17  4  2 23 33  5 44 23 43]
 [47 13 17 17 79 29 73 23  4 73]
 [23 16 73 73  4 23 33  4 68 23]
 [66 14 77 29 73 17 51 78 24 23]]

y
 [[33  4 39 43  5 44 23 32 54 54]
 [29 73 23 43 33  4 43 23  4 43]
 [73 68 23 29 44 23  4 23  3 29]
 [23 43 33  5 23 47 33 79  5  3]
 [17  4  2 23 33  5 44 23 43  5]
 [13 17 17 79 29 73 23  4 73 68]
 [16 73 73  4 23 33  4 68 23 17]
 [14 77 29 73 17 51 78 24 23 52]]


Если вы правильно реализовали get_batches, результат должен выглядеть примерно так:
```
x
 [[25  8 60 11 45 27 28 73  1  2]
 [17  7 20 73 45  8 60 45 73 60]
 [27 20 80 73  7 28 73 60 73 65]
 [17 73 45  8 27 73 66  8 46 27]
 [73 17 60 12 73  8 27 28 73 45]
 [66 64 17 17 46  7 20 73 60 20]
 [73 76 20 20 60 73  8 60 80 73]
 [47 35 43  7 20 17 24 50 37 73]]

y
 [[ 8 60 11 45 27 28 73  1  2  2]
 [ 7 20 73 45  8 60 45 73 60 45]
 [20 80 73  7 28 73 60 73 65  7]
 [73 45  8 27 73 66  8 46 27 65]
 [17 60 12 73  8 27 28 73 45 27]
 [64 17 17 46  7 20 73 60 20 80]
 [76 20 20 60 73  8 60 80 73 17]
 [35 43  7 20 17 24 50 37 73 36]]
 ```
 хотя точные цифры могут отличаться. Убедитесь, что данные сдвинуты на один шаг для `y`!!!

---
## Defining the network with PyTorch


<img src="https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/assets/charRNN.png?raw=1" width=500px>

Затем используем PyTorch для определения архитектуры сети. Начнем с определения слоев и операций, методов прямого прохода. 

### Model Structure

В `__init__` предлагаемая структура выглядит следующим образом:
* Создавать и хранить необходимые словари (это было сделано за вас)
* Определите слой LSTM, который принимает в качестве параметров: размер ввода (количество символов), размер скрытого слоя `n_hidden`, количество слоев` n_layers`, вероятность выпадения `drop_prob` и логическое значение batch_first (True)
* Определите слой отброса данных с помощью drop_prob
* Определите полносвязанный слой с параметрами: размер ввода `n_hidden` и размер выхода - количество символов
* Наконец, инициализируйте веса

Обратите внимание, что некоторые параметры были названы и указаны в функции `__init__`, их нужно сохранить и использовать, выполняя что-то вроде` self.drop_prob = drop_prob`.

---
### LSTM Inputs/Outputs

Вы можете создать [LSTM layer](https://pytorch.org/docs/stable/nn.html#lstm) следующим образом

```python
self.lstm = nn.LSTM(input_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
```

где input_size - это количество символов, которые эта ячейка ожидает видеть в качестве последовательного ввода, а n_hidden - это количество единиц в скрытых слоях ячейки. Можно добавить выпадение, добавив параметр выпадения с заданной вероятностью; это автоматически добавит отсев на входах или выходах. Наконец, в функции `forward` мы можем складывать ячейки LSTM в слои, используя `.view`. При этом он отправляет вывод одной ячейки в следующую ячейку.

Здесь же требуется создать начальное скрытое состояние всех нулей:

```python
self.init_hidden()
```

In [11]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [12]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(#YOUR CODDE HERE,
                            n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(
            # YOUR CODE HERE
        )
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = ...
        
        ## TODO: pass through a dropout layer
        out = ...
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## TODO: put x through the fully-connected layer
        out = ...
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

## Time to train

Во время обучения нужно установить количество эпох, скорость обучения и другие параметры.

Используем оптимизатор Адама и кросс-энтропию, считаем loss и, как обычно, выполняем back propagation!

Пара подробностей об обучении:
> * В рамках цикла мы отделяем скрытое состояние от его истории; на этот раз установив его равным новой переменной * tuple *, потому что LSTM имеет скрытое состояние, которое является кортежем скрытых состояний.
* Мы используем [`clip_grad_norm_`](https://pytorch.org/docs/stable/_modules/torch/nn/utils/clip_grad.html) чтобы избавиться от взрывающегося градиента.

In [13]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            if counter == 2:
                print(inputs.shape, targets.shape)
                print(inputs, targets)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

## Instantiating the model

Теперь мы можем создать модель с заданными гиперпараметрами. Определим размеры мини-батчей!

In [14]:
# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


### Set your training hyperparameters!

In [15]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

torch.Size([128, 100, 83]) torch.Size([128, 100])
tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0.,

## Getting the best model

Чтобы настроить гиперпараметры на максимальную производительность, вам нужно будет посмотреть потери при обучении и проверке. Если ваша потеря на обучении намного ниже, чем потеря на тесте, то модель переобучена. Увеличьте регуляризацию или уменьшите число слоев в сети.

В ячейке ниже вы можете увидеть рекомендации, которые были найдены в статьях по оптимизации RNN.

## Hyperparameters

Гиперпараметры сети:

* `n_hidden` - Количество юнитов в скрытых слоях.
* `n_layers` - Количество используемых скрытых слоев LSTM.

В нашем примере вероятность отсева и скорость обучения сохраняется.

Для обучения:
* `batch_size` - количество объектов в батче, проходящих по сети за один проход.
* `seq_length` - Количество символов в последовательности, на которой обучается сеть. Обычно чем больше, тем лучше, сеть будет изучать более дальние зависимости.
* `lr` - learning rate.


 ## Советы и хитрости

> -  В глубоком обучении очень распространено запускать множество различных моделей с множеством различных настроек гиперпараметров и, в конце концов, использовать любую контрольную точку, дающую наилучшую производительность проверки.

> - Кстати, размер ваших тренировочных и проверочных разделов также является параметрами. Убедитесь, что у вас есть приличный объем данных в вашем наборе проверки, иначе производительность проверки будет шумной и не очень информативной.

## Checkpoint

После обучения сохраним модель, чтобы можно было загрузить ее позже. Здесь сохраняются параметры, необходимые для создания той же архитектуры, гиперпараметры скрытого слоя и токены.

In [29]:
# change the name, for saving multiple files
model_name = 'rnn_x_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

---
## Making Predictions

Теперь, когда модель обучена, сделаем предсказание следующих символов! Для предсказания мы передаем последний символ обучения, и сеть предсказывает следующий символ, который мы потом передаем обратно и получаем еще один предсказанный символ и так далее...

Наши прогнозы основаны на категориальном распределении вероятностей по всем возможным символам. Мы можем ограничить число символов, чтобы сделать получаемый предсказанный текст более разумным, рассматривая только некоторые наиболее вероятные символы $K$. Это не позволит сети выдавать нам совершенно абсурдные символы, а также позволит внести некоторый шум и случайность в выбранный текст. Узнать больше [можно здесь](https://pytorch.org/docs/stable/torch.html#torch.topk).


In [30]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

### Priming and generating text 

Нужно создать скрытое состояние, чтобы сеть не генерировала произвольные символы. 

In [31]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [32]:
print(sample(net, 1000, prime='Anna', top_k=5))

Anna had now to say, and he was there it was a past. At that she was thinking of his boy and always back at that sort. There was a cannoticater immediate
all to her
this stream.

Seryozha saw that he had been coughing their conversation and the stepan or anyone
with the
promited, and
she sent him and starting, with his figure at the moment of
his face, before.

Sergey Ivanovitch with another children with the part were being set all asked, without taking off his stay.

Stepan Arkadyevitch's well so talked that he sat down at the
most frillly weary, before her excitement when she said, when she went on the same sociecy,
was the passions of a last pluch. At the cold matters and head with socioting prince. "I'll go in and stirll to me..."

"What don't be the barty of his? And its difeering at the comprehonse of the same man of her house," he said at the room,
saying the solidiculies to
him to stand out in the corried, his hands of his hand, a look, and to think of a performance, which was

## Loading a checkpoint

In [ ]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_x_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

In [ ]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="And Levin said"))